Name: Tchangmena A Nken Allassan

creation date: 30/10/2021

update date:   30/10/2021


we use  GPU to train this model

In [1]:
pip install -q git+https://github.com/eaedk/testing-zindi-package.git

In [2]:
# Import Zindian class from the package
from zindi.user import Zindian

In [3]:
# create a user object
my_username = "Tony_Stark"
user = Zindian(username = my_username)

Your password
>> ··········

[ 🟢 ] 👋🏾👋🏾 Welcome Tony_Stark 👋🏾👋🏾



In [4]:
user.select_a_challenge()                               # Select a Zindi challenge
user.which_challenge                                    # Get information about the selected challenge

__________________________________________________________________________________________________________________________________
|     |              |                  |                    |          
|index|  challenge   |     problem      |       reward       |    id    
|     |              |                  |                    |          
----------------------------------------------------------------------------------------------------------------------------------
|  0  | Public Hack  |                  |        $800        | coral-classification-challenge...
----------------------------------------------------------------------------------------------------------------------------------
|  1  |Public Compet |  Classification  |     $7 500 USD     | bloodsai-blood-spectroscopy-classification-challen...
----------------------------------------------------------------------------------------------------------------------------------
|  2  |Public Compet |Automatic Speech R|  

'coral-classification-challenge'

In [5]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

dataset/Train.csv: 100%|██████████| 537k/537k [00:01<00:00, 491ko/s]
dataset/SampleSubmission.csv: 100%|██████████| 210k/210k [00:00<00:00, 243ko/s]
dataset/Test.csv: 100%|██████████| 199k/199k [00:00<00:00, 232ko/s]
dataset/coral_image.zip: 100%|██████████| 117M/117M [00:19<00:00, 6.16Mo/s]


## Data Loading

In [6]:
#import useful packages
import numpy as np
import pandas as pd
import os

In [7]:
path='dataset/'

In [8]:
train_csv=pd.read_csv(path+'Train.csv')
test_csv=pd.read_csv(path+'Test.csv')

In [10]:
#!unzip 'dataset/coral_image.zip'

In [11]:
train_csv.label.value_counts()

Algae                  11530
Hard Coral              6067
Other                   1571
Soft Coral              1553
Other Invertebrates      959
Name: label, dtype: int64

The images have imbalanced classes I will try to do data augmentation on the lower classes to counter balance the higher ones

## Preprocessing and data Augmentation 

In [12]:
train_csv.head()

,Image_ID,label
0,ImageID_001G9EFI,Other
1,ImageID_0026LXMN,Hard Coral
2,ImageID_004J6Y6A,Hard Coral
3,ImageID_0056CP11,Hard Coral
4,ImageID_00AV0G9B,Other


In [13]:
#making directories
os.mkdir('Train')
os.mkdir('Test')

In [14]:
os.mkdir('Train/Algae')
os.mkdir('Train/Hard_Coral')
os.mkdir('Train/Other')
os.mkdir('Train/Soft_Coral')
os.mkdir('Train/Other_Invertebrates')

In [15]:
list_algae=list(train_csv[train_csv.label=="Algae"].Image_ID)
list_hard_coral=list(train_csv[train_csv.label=="Hard Coral"].Image_ID)
list_other=list(train_csv[train_csv.label=="Other"].Image_ID)
list_Soft_coarl=list(train_csv[train_csv.label=="Soft Coral"].Image_ID)
list_other_inver=list(train_csv[train_csv.label=="Other Invertebrates"].Image_ID)

In [16]:
list_test=list(test_csv.Image_ID)

In [17]:
## We start by moving the classes to their respective directories

def move_file(old_file_path, new_directory):
    if not os.path.isdir(new_directory):
        os.mkdir(new_directory)
    base_name = os.path.basename(old_file_path)
    new_file_path = os.path.join(new_directory, base_name)
    # Deletes a file if that file already exists there, you can change this behavior
    if os.path.exists(new_file_path):
        os.remove(new_file_path)
    os.rename(old_file_path, new_file_path)


In [23]:
#lets move algaes,coral,invertabrates....to their respective folders

cop_folder = 'coral_image/'

destination_folder = 'Train/Soft_Coral/'


for i in list_Soft_coarl:
    filename = os.path.join(cop_folder, i+'.jpg')
    move_file(filename, destination_folder)

## Augmentation in Other

In [39]:
import os
import random
from scipy import ndarray

# image processing library
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io

def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise,
    'horizontal_flip': horizontal_flip
}

folder_path = 'Train/Other_Invertebrates/'
num_files_desired = 10000

# find all files paths from the folder
images = [x+".jpg" for x in list_other_inver]

num_generated_files = 0
while num_generated_files <= num_files_desired:
    # random image from the folder
    image_path = random.choice(images)
    # read image as an two dimensional array of pixels
    image_to_transform = sk.io.imread(folder_path+image_path)
    # random num of transformation to apply
    num_transformations_to_apply = random.randint(1, len(available_transformations))

    num_transformations = 0
    transformed_image = None
    while num_transformations <= num_transformations_to_apply:
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](image_to_transform)
        num_transformations += 1

        new_file_path = '%s/augmented_image_%s.jpg' % (folder_path, num_generated_files)

        # write image to the disk
        io.imsave(new_file_path, transformed_image)
    num_generated_files += 1

Streaming output truncated to the last 5000 lines.


In [41]:
print(len(os.listdir('Train/Soft_Coral')))
print(len(os.listdir('Train/Hard_Coral')))
print(len(os.listdir('Train/Algae')))
print(len(os.listdir('Train/Other_Invertebrates')))
print(len(os.listdir('Train/Other')))

10554
11068
11530
10960
10572


In [42]:
from shutil import make_archive

make_archive('Test_coral', 'zip', root_dir='Test')

'/content/Test_coral.zip'

In [43]:
make_archive('Train_coral', 'zip', root_dir='Train')

'/content/Train_coral.zip'